In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
!pip install geopy
from geopy.geocoders import Nominatim
!pip install folium
import folium
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
print("Libraries installed")

     |████████████████████████████████| 102kB 7.0MB/s ta 0:00:011
Libraries installed


# Question 1

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(result.headers)

{'Date': 'Thu, 11 Jun 2020 15:35:04 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.7', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'en', 'Last-Modified': 'Mon, 01 Jun 2020 18:03:48 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '50202', 'X-Cache': 'cp1081 miss, cp1087 hit/144', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=12-Jun-2020;Path=/;HttpOnly;secure;Expires=Tue, 14 Jul 2020 00:00:00 GMT, WMF-Last-Access-Global=12-Jun-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Tue, 14 Jul 2020 00:00:00 GMT, GeoIP=GB:ENG:London:51.50:-0.22:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '158.176.106.114', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate

Import web page and parsing using BeatifulSoup

In [3]:
soup = BeautifulSoup(result.content, 'html.parser')

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"39267423-2d13-465f-959d-7a5982534bc3","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

Find the tag with table to extract values

In [5]:
Table=soup.find('table')

Preparing the dataframe

In [6]:
Toronto=pd.DataFrame(columns=['Postal Code','Borough','Neighbourhood'])
rows=Table.find_all('tr')

for i in range(1,len(rows)):
    row_values=rows[i].find_all('td')
    list_row_values=[]
    for j in range(0,len(row_values)):
        var=row_values[j].get_text().rstrip("\n")
        #print(var)
        list_row_values.append(var)
    #print(list_row_values)
    Toronto = Toronto.append(pd.Series(list_row_values, index=['Postal Code','Borough','Neighbourhood']),ignore_index=True)
       
Toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Dropping rows without any Borough value

In [7]:
indexNames=Toronto[Toronto['Borough']=='Not assigned'].index
Toronto.drop(indexNames , inplace=True)

In [8]:
Toronto.shape

(103, 3)

Assigning the borough name to the Neighbourhood having no values

In [9]:
Toronto.loc[Toronto['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = Toronto.loc[Toronto['Neighbourhood'] == 'Not assigned',['Borough']]
Toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Printing the shape of the Dataframe

In [10]:
Toronto.shape

(103, 3)

# Question 2

Import geocode data

In [11]:
geocode_link='http://cocl.us/Geospatial_data'
Geocode_df=pd.read_csv(geocode_link)
Geocode_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge latitude longitude data into Toronto dataframe

In [12]:
Toronto_Geo_code=Toronto.merge(Geocode_df,on='Postal Code',how='inner')

In [13]:
Toronto_Geo_code.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Question 3

Let us select Borough North York

In [14]:
Borough_NorthYork=Toronto_Geo_code[Toronto_Geo_code['Borough']=='North York']
Borough_NorthYork.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M3B,North York,Don Mills,43.745906,-79.352188
10,M6B,North York,Glencairn,43.709577,-79.445073


Creating CLIENT ID and CLIENT SECRET for later use

In [15]:
CLIENT_ID = 'VHHOWATJ542UNDLKMWZXDCD1KM4XNQKLDKKH2X03G02H4SXK' # your Foursquare ID
CLIENT_SECRET = 'OT2BBQA0BWP5P5ULVPRIWJS4H5O5WFSAR02FLX0XPIGEL0ZN' # your Foursquare Secret
VERSION = '20200610' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VHHOWATJ542UNDLKMWZXDCD1KM4XNQKLDKKH2X03G02H4SXK
CLIENT_SECRET:OT2BBQA0BWP5P5ULVPRIWJS4H5O5WFSAR02FLX0XPIGEL0ZN


Getting venue category details

In [16]:
def venue_category_details(latitude,longitude,neighbourhood):
    LIMIT = 100
    radius = 500
    lat=latitude
    long=longitude
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
    result=requests.get(url).json()
    result_list=result['response']['groups'][0]['items']
    Venue_df=pd.DataFrame(columns=['Neighbourhood','Venue Name','Venue Category','Venue ID','Latitude','Longitude'])
    for i in range(0,len(result_list)):
        try:
            venue_id=result_list[i]['venue']['id']
            venue_name=result_list[i]['venue']['name']
            venue_category=result_list[i]['venue']['categories'][0]['name']
            Venue_df=Venue_df.append({'Neighbourhood':neighbourhood,'Venue Name':venue_name,'Venue Category':venue_category,'Venue ID':venue_id,'Latitude':lat,'Longitude':long},ignore_index=True)
        except:
            pass
    
    return Venue_df

In [17]:
Venues_NY=pd.DataFrame(columns=['Neighbourhood','Venue Name','Venue Category','Venue ID','Latitude','Longitude'])

for i in range(0,Borough_NorthYork.shape[0]):
    neighbourhood=Borough_NorthYork.iloc[i,2]
    lat=Borough_NorthYork.iloc[i,3]
    long=Borough_NorthYork.iloc[i,4]
    result=venue_category_details(lat,long,neighbourhood)
    Venues_NY=Venues_NY.append(result,ignore_index=True)
Venues_NY.head()


,Neighbourhood,Venue Name,Venue Category,Venue ID,Latitude,Longitude
0,Parkwoods,Brookbanks Park,Park,4e8d9dcdd5fbbbb6b3003c7b,43.753259,-79.329656
1,Parkwoods,GTA Restoration,Fireworks Store,53e72d92498e336f61db5bf5,43.753259,-79.329656
2,Parkwoods,Variety Store,Food & Drink Shop,4cb11e2075ebb60cd1c4caad,43.753259,-79.329656
3,Victoria Village,Victoria Village Arena,Hockey Arena,4c633acb86b6be9a61268e34,43.725882,-79.315572
4,Victoria Village,Tim Hortons,Coffee Shop,4bbe904a85fbb713420d7167,43.725882,-79.315572


In [18]:
Venues_NY.shape

(241, 6)

In [19]:
Venues_NY.groupby('Neighbourhood').count()

,Venue Name,Venue Category,Venue ID,Latitude,Longitude
Neighbourhood,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20
Bayview Village,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24
Don Mills,28,28,28,28,28
Downsview,17,17,17,17,17
"Fairview, Henry Farm, Oriole",62,62,62,62,62
Glencairn,6,6,6,6,6
Hillcrest Village,5,5,5,5,5
Humber Summit,2,2,2,2,2


One hot encoding of the venue types

In [20]:
Venues_NY_onehot = pd.get_dummies(Venues_NY[['Venue Category']],prefix="", prefix_sep="")
Venues_NY_onehot['Neighbourhood'] = Venues_NY['Neighbourhood']
cols = Venues_NY_onehot.columns.tolist()
cols=cols[-1:]+cols[:-1]
Venues_NY_onehot=Venues_NY_onehot[cols]
Venues_NY_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
Venues_NY_mean=Venues_NY_onehot.groupby('Neighbourhood').mean().reset_index()
Venues_NY_mean

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.035714,0.000000,0.071429,0.035714,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.016129,0.000000,0.000000,0.016129,0.000000,0.032258,0.032258,...,0.016129,0.000000,0.016129,0.000000,0.016129,0.016129,0.0,0.016129,0.000000,0.032258
6,Glencairn,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,...,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


Getting top five venues for each neighbourhood

In [22]:
Venues_NY_mean_final=pd.DataFrame(columns=['Neighbourhood','1st venue category','2nd venue category','3rd venue category','4th venue category','5th venue category'])
Col_list=list(Venues_NY_mean.columns)
#print(Col_list)
for i in range(0,Venues_NY_mean.shape[0]):
    Col_list_top_category=[Venues_NY_mean.iloc[i,0]]
    temp_list=[]
    row_list=list(Venues_NY_mean.iloc[i,:])
    temp_df=pd.DataFrame(list(zip(Col_list[1:],row_list[1:])), 
               columns =['col names', 'val'])
    Top_5_temp_df=temp_df.sort_values(by='val',ascending=False).head()
    Col_list_top_category.extend(list(Top_5_temp_df['col names']))
    temp_list=Col_list_top_category
    temp_series = pd. Series(temp_list, index = Venues_NY_mean_final. columns)
    Venues_NY_mean_final=Venues_NY_mean_final.append(temp_series,ignore_index=True)
Venues_NY_mean_final.head()

,Neighbourhood,1st venue category,2nd venue category,3rd venue category,4th venue category,5th venue category
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Bridal Shop,Ice Cream Shop,Gas Station
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Accessories Store
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Greek Restaurant
3,Don Mills,Sporting Goods Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Coffee Shop
4,Downsview,Grocery Store,Business Service,Park,Korean Restaurant,Discount Store


Cluster Neighbourhood

In [32]:
# set number of clusters
kclusters = 5

Venues_NY_clustering = Venues_NY_mean.drop('Neighbourhood', 1)
Venues_NY_clustering
# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]

,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,...,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.035714,0.000000,0.071429,0.035714,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.016129,0.000000,0.000000,0.016129,0.000000,0.032258,0.032258,0.016129,...,0.016129,0.000000,0.016129,0.000000,0.016129,0.016129,0.0,0.016129,0.000000,0.032258
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


Using k-means clustering for labeling each neighbourhood

In [34]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Venues_NY_clustering)
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 3, 1, 3, 1, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [45]:
# add clustering labels
Venues_NY_mean_final.insert(0, 'Cluster Labels', kmeans.labels_)
#Venues_NY_mean_final.head()


In [39]:
Venues_NY_mean_final.head()

,Cluster Labels,Neighbourhood,1st venue category,2nd venue category,3rd venue category,4th venue category,5th venue category
0,3,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Bridal Shop,Ice Cream Shop,Gas Station
1,3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Accessories Store
2,3,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Greek Restaurant
3,3,Don Mills,Sporting Goods Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Coffee Shop
4,3,Downsview,Grocery Store,Business Service,Park,Korean Restaurant,Discount Store


In [40]:
Venues_NY_final_df=Venues_NY
Venues_NY_final_df = Venues_NY_final_df.join(Venues_NY_mean_final.set_index('Neighbourhood'), on='Neighbourhood')

In [41]:
Venues_NY_final_df.head()

,Neighbourhood,Venue Name,Venue Category,Venue ID,Latitude,Longitude,Cluster Labels,1st venue category,2nd venue category,3rd venue category,4th venue category,5th venue category
0,Parkwoods,Brookbanks Park,Park,4e8d9dcdd5fbbbb6b3003c7b,43.753259,-79.329656,0,Park,Food & Drink Shop,Fireworks Store,Korean Restaurant,Movie Theater
1,Parkwoods,GTA Restoration,Fireworks Store,53e72d92498e336f61db5bf5,43.753259,-79.329656,0,Park,Food & Drink Shop,Fireworks Store,Korean Restaurant,Movie Theater
2,Parkwoods,Variety Store,Food & Drink Shop,4cb11e2075ebb60cd1c4caad,43.753259,-79.329656,0,Park,Food & Drink Shop,Fireworks Store,Korean Restaurant,Movie Theater
3,Victoria Village,Victoria Village Arena,Hockey Arena,4c633acb86b6be9a61268e34,43.725882,-79.315572,3,Grocery Store,Hockey Arena,Portuguese Restaurant,Coffee Shop,Lounge
4,Victoria Village,Tim Hortons,Coffee Shop,4bbe904a85fbb713420d7167,43.725882,-79.315572,3,Grocery Store,Hockey Arena,Portuguese Restaurant,Coffee Shop,Lounge


Finally, let's visualize the resulting clusters

In [43]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Venues_NY_final_df['Latitude'], Venues_NY_final_df['Longitude'], Venues_NY_final_df['Neighbourhood'], Venues_NY_final_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters